In [1]:
!pip install pennylane
!pip install transformers
!pip install scikit-learn
!pip install tensorflow
!pip install tensorflow-datasets

  Using cached PennyLane-0.31.1-py3-none-any.whl (1.5 MB)
  Using cached PennyLane_Lightning-0.31.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.6 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached autograd-1.5-py3-none-any.whl (48 kB)
  Using cached autoray-0.6.3-py3-none-any.whl (48 kB)

[notice] A new release of pip available: 23.1.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached regex-2023.8.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached safetensors-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)

[notic

In [2]:
import pennylane as qml
import torch
from torch import Tensor
import numpy as np
import torch.nn as nn

from tqdm.notebook import tqdm

from transformers import AutoTokenizer

from torch.utils.data import DataLoader, Dataset
from tensorflow import compat
import tensorflow_datasets as tfds
import scipy

from pathlib import Path
import pandas as pd
from joblib import load, dump
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader, Dataset
import torch.utils.data as data_utils

from collections import Counter

2023-08-13 11:08:47.913781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 11:08:49.212587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Model

In [3]:
import functools
import inspect
import math
from collections.abc import Iterable
from typing import Callable, Dict, Union, Any

from pennylane.qnode import QNode

try:
    import torch
    from torch.nn import Module

    TORCH_IMPORTED = True
except ImportError:
    # The following allows this module to be imported even if PyTorch is not installed. Users
    # will instead see an ImportError when instantiating the TorchLayer.
    from unittest.mock import Mock

    Module = Mock
    TORCH_IMPORTED = False


class TorchLayer(Module):
    def __init__(self,qnode,weights):
        if not TORCH_IMPORTED:
            raise ImportError(
                "TorchLayer requires PyTorch. PyTorch can be installed using:\n"
                "pip install torch\nAlternatively, "
                "visit https://pytorch.org/get-started/locally/ for detailed "
                "instructions."
            )
        super().__init__()

        #weight_shapes = {
        #    weight: (tuple(size) if isinstance(size, Iterable) else () if size == 1 else (size,))
        #    for weight, size in weight_shapes.items()
        #}

        # validate the QNode signature, and convert to a Torch QNode.
        # TODO: update the docstring regarding changes to restrictions when tape mode is default.
        #self._signature_validation(qnode, weight_shapes)
        self.qnode = qnode
        self.qnode.interface = "torch"

        self.qnode_weights = weights

    def forward(self, inputs):  # pylint: disable=arguments-differ
        """Evaluates a forward pass through the QNode based upon input data and the initialized
        weights.

        Args:
            inputs (tensor): data to be processed

        Returns:
            tensor: output data
        """

        if len(inputs.shape) > 1:
            # If the input size is not 1-dimensional, unstack the input along its first dimension,
            # recursively call the forward pass on each of the yielded tensors, and then stack the
            # outputs back into the correct shape
            reconstructor = [self.forward(x) for x in torch.unbind(inputs)]
            return torch.stack(reconstructor)

        # If the input is 1-dimensional, calculate the forward pass as usual
        return self._evaluate_qnode(inputs)


    def _evaluate_qnode(self, x):
        """Evaluates the QNode for a single input datapoint.

        Args:
            x (tensor): the datapoint

        Returns:
            tensor: output datapoint
        """
        kwargs = {
            **{self.input_arg: x},
            **{arg: weight.to(x) for arg, weight in self.qnode_weights.items()},
        }
        res = self.qnode(**kwargs)

        if isinstance(res, torch.Tensor):
            return res.type(x.dtype)

        return torch.hstack(res).type(x.dtype)

    def __str__(self):
        detail = "<Quantum Torch Layer: func={}>"
        return detail.format(self.qnode.func.__name__)

    __repr__ = __str__
    _input_arg = "inputs"

    @property
    def input_arg(self):
        """Name of the argument to be used as the input to the Torch layer. Set to ``"inputs"``."""
        return self._input_arg

In [4]:
class qrnn(torch.nn.Module):

    def __init__(self,S,q_n_h,q_n_d,D,voc_size=20):
        """
        # input: input data
        # weight: trainable parameter
        # q_n_h: # of of qubits for encodong input
        # q_n_d: # of of qubits for memerizing history
        # d: embedding dimension which is equal to q_n_h*(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        """
        super().__init__()
        self.voc_size=voc_size
        self.seq_num=S
        self.num_q=q_n_h+q_n_d
        self.q_n_d=q_n_d
        self.q_n_h=q_n_h
        self.init_params=torch.nn.Parameter((np.pi/4) * (2 * torch.randn(self.num_q*(3*D+2)*self.seq_num) - 1))

        self.D=D
        self.d=q_n_h
        self.params_num_per_lyr=self.num_q*(3*D+2)

        self.dev = qml.device("default.qubit", wires=self.num_q)
        self.qnod=qml.QNode(self.circuit, self.dev, interface="torch")
        self.weight = {"weights": self.init_params}
        self.QRNN = TorchLayer(self.qnod, self.weight)

    def circuit(self,inputs,weights):

        for sq in range(self.seq_num):
            start=sq*self.d
            end=(sq+1)*self.d
            self.Feature_map(inputs[start:end])
            start=sq*self.params_num_per_lyr
            end=(sq+1)*self.params_num_per_lyr
            self.ansatz(self.init_params[start:end])

        return qml.expval(qml.PauliZ(0))

    def ansatz(self,weights):
        indx=0
        for j in range(self.num_q):
            qml.RX(weights[indx],wires=j)
            qml.RZ(weights[indx+1],wires=j)
            qml.RX(weights[indx+2],wires=j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qml.IsingZZ(weights[indx],wires=[j,(j+1)%self.num_q])
                #qml.CNOT(wires=[j,(j+1)%self.num_q])
                #qml.RZ(weights[indx],wires=(j+1)%self.num_q)
                #qml.CNOT(wires=[j,(j+1)%self.num_q])
                indx+=1
            for j in range(self.num_q):
                #qml.RX(weights[indx],wires=j)
                qml.RY(weights[indx],wires=j)
                indx+=1

    def Feature_map(self, input):
        indx=0
        for j in range(self.q_n_h):
            qml.RY(input[indx],j+self.q_n_d)

    def forward(self, x):

        x=self.QRNN(x)
        return torch.sigmoid(x)

# MC RP DATASET

In [3]:
!git clone https://github.com/CQCL/qnlp_lorenz_etal_2021_resources
!mv qnlp_lorenz_etal_2021_resources/datasets mc_rp_dataset

fatal: destination path 'qnlp_lorenz_etal_2021_resources' already exists and is not an empty directory.
mv: cannot stat 'qnlp_lorenz_etal_2021_resources/datasets': No such file or directory


In [4]:
mc_rp_sets_path = Path("mc_rp_dataset")
mc_rp_sets = list(mc_rp_sets_path.glob("*.txt"))
mc_datasets, rp_datasets = list(filter(lambda x: x.name.startswith("mc"), mc_rp_sets)), list(filter(lambda x: x.name.startswith("rp"), mc_rp_sets))

In [5]:
def read_process_mcrp(datapaths: dict):
  def rm(text):
    return " ".join(list(map(lambda x: x[:x.find('_')], text.split())))
  retval = {}
  for datapath in datapaths:
    if "rp" in str(datapath):
      sel = 2
    else:
      sel = 3
    df = pd.DataFrame(list(map(lambda x: [int(x[0]), x[sel:]], datapath.read_text().split("\n"))), columns=['label', 'text'])
    df['text'] = df['text'].apply(rm)
    retval[datapath.name.split(".")[0]] = df
  return retval

mc_data, rp_data = read_process_mcrp(mc_datasets), read_process_mcrp(rp_datasets)
MRP_BATCH_SIZE = 30

In [6]:
def data_frame_simple_txt2vec(data,keystr,shape,min_length=4,w_dict=None):
    txt_lst=[]
    max_len=0
    for txt in data[keystr].text:
        txtsplit=txt.split()
        txt_lst=txt_lst+txtsplit
        if max_len< len(txtsplit):
            max_len=len(txtsplit)
    w_set=set(txt_lst)

    comm_lst=Counter(txt_lst).most_common()

    if w_dict==None:
        w_dict={}
        ind=len(comm_lst)
        for elem in comm_lst:
            w_dict[elem[0]]=ind
            ind-=1
    #w_dict={}
    #ind=1
    #for elem in w_set:
    #    w_dict[elem]=ind
    #    ind+=1

    if max_len%2 !=0:
        max_len+=1
    if max_len<min_length:
        max_len=min_length

    label_list=[]
    Txt_list=[]
    for i, txt in enumerate(data[keystr].text):
        w_list=[]
        for word in data[keystr].text[i].split():
            if word in w_dict:
                w_list.append(w_dict[word])
            else:
                w_list.append(len(w_dict.keys())+1)
        if len(w_list) < max_len:
            w_list=w_list+(max_len-len(w_list))*[0]
        #print(torch.tensor(w_list))
        label_list.append(torch.tensor(data[keystr].label[i]))
        Txt_list.append(torch.tensor(w_list).reshape(shape)*np.pi/len(comm_lst)/2)
    label_tensor=torch.tensor(label_list)
    txt_tensor=torch.stack(Txt_list)

    return data_utils.TensorDataset(txt_tensor,label_tensor),w_dict

## MC Task

In [7]:
shape=(4)

mc_df_train,train_dict=data_frame_simple_txt2vec(mc_data,'mc_train_data',shape)
mc_df_test,_=data_frame_simple_txt2vec(mc_data,'mc_test_data',shape,w_dict=train_dict)

mc_trainloader = DataLoader(mc_df_train, shuffle=True, batch_size=30)
mc_testloader = DataLoader(mc_df_test, shuffle=True, batch_size=30)

In [10]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [11]:
model = qrnn(4,1,3,2)

In [12]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

128


In [13]:
n_epochs=300
optimizer = torch.optim.Adam(lr=0.008, params=model.parameters())
criterion = nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()
#pbar = tqdm(total=len(mc_trainloader), leave=True)
if torch.cuda.is_available():
    devname = "cuda"
else:
    devname = "cpu"
device = torch.device(devname)
model.train()
model.to(device)
#pbar = tqdm(total=len(mc_trainloader), leave=True)
file_name="QRNN_model_12111_mc"
max_val=0
loss_lst=[]
for epoch in range(n_epochs):
    acc=0
    aver_lst=[]
    for batch, (feature, label) in enumerate(mc_trainloader):
        optimizer.zero_grad()
        feature, label = feature.to(device), label.to(device)
        preds = model(feature.float())#.squeeze(1)
        loss = criterion(preds.flatten(), label.float())
        preds = preds.ravel()
        label = label.ravel()
        acc += binary_accuracy(preds, label)

        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch} | Batch: {batch} | Loss {loss.cpu().detach()}")
        #pbar.update()
        #pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss.cpu().detach()}"
        #pbar.refresh()
        aver_lst.append(loss.cpu().detach())
    loss_lst.append(np.mean(aver_lst))
    acc=acc/(batch+1)
    print('Epoch',epoch,'Training Accuracy:', acc)
    feature, label=next(iter(mc_testloader))
    preds = model(feature.float().squeeze(1))#.squeeze(1)
    loss = criterion(preds.flatten(), label.float())
    preds = preds.ravel()
    label = label.ravel()
    acc1 = binary_accuracy(preds, label)
    print('Testing Accuracy:', acc1)
    if max_val< (acc1+acc):
        max_val=acc1+acc
        torch.save(model.state_dict(),file_name)
#pbar.close()


Epoch: 0 | Batch: 0 | Loss 0.6494042277336121
Epoch: 0 | Batch: 1 | Loss 0.6984403729438782
Epoch: 0 | Batch: 2 | Loss 0.7219951748847961
Epoch 0 Training Accuracy: tensor(0.5222)
Testing Accuracy: tensor(0.5000)
Epoch: 1 | Batch: 0 | Loss 0.6711748838424683
Epoch: 1 | Batch: 1 | Loss 0.6526397466659546
Epoch: 1 | Batch: 2 | Loss 0.6134403347969055
Epoch 1 Training Accuracy: tensor(0.5889)
Testing Accuracy: tensor(0.5000)
Epoch: 2 | Batch: 0 | Loss 0.6534045934677124
Epoch: 2 | Batch: 1 | Loss 0.6310886144638062
Epoch: 2 | Batch: 2 | Loss 0.5810405015945435
Epoch 2 Training Accuracy: tensor(0.6333)
Testing Accuracy: tensor(0.6333)
Epoch: 3 | Batch: 0 | Loss 0.6330143809318542
Epoch: 3 | Batch: 1 | Loss 0.601358950138092
Epoch: 3 | Batch: 2 | Loss 0.6109526753425598
Epoch 3 Training Accuracy: tensor(0.7778)
Testing Accuracy: tensor(0.7000)
Epoch: 4 | Batch: 0 | Loss 0.6434726119041443
Epoch: 4 | Batch: 1 | Loss 0.5826245546340942
Epoch: 4 | Batch: 2 | Loss 0.5399240255355835
Epoch 4 Tra

In [13]:
file_name="QRNN_model_12111_mc"
model.load_state_dict(torch.load(file_name))
model.eval()

qrnn(
  (QRNN): <Quantum Torch Layer: func=circuit>
)

In [14]:
acc=[]
for i in range(100):
    preds = []
    labels = []
    #pbar = tqdm(total=len(mc_testloader), leave=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    for batch, (feature, label) in enumerate(mc_testloader):
      feature, label = feature.to(device), label.to(device)
      with torch.no_grad():
        predictions = model(feature.float())
        preds.append(predictions.cpu().numpy())
        labels.append(label.cpu().numpy())
      #pbar.update()
      #pbar.desc = f"Batch: {batch}"
      #pbar.refresh()
    #pbar.close()
    preds = torch.tensor(preds).ravel()
    # preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
    labels = torch.tensor(labels).ravel()
    #acc = (preds == labels).sum() / len(preds)
    acc.append(binary_accuracy(preds, labels))
print("Test Accuracy:",np.mean(acc),r"$\pm$",np.std(acc))

/tmp/ipykernel_4020/4241768363.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  preds = torch.tensor(preds).ravel()


Test Accuracy: 0.7333333 $\pm$ 5.9604645e-08


## RP task

In [15]:
shape=(4)

rp_df_train,train_dict=data_frame_simple_txt2vec(rp_data,'rp_train_data',shape)
rp_df_test,_=data_frame_simple_txt2vec(rp_data,'rp_test_data',shape,w_dict=train_dict)

rp_trainloader = DataLoader(rp_df_train, shuffle=True, batch_size=30)
rp_testloader = DataLoader(rp_df_test, shuffle=True, batch_size=31)

In [17]:
model_2 = qrnn(4,1,1,2)

In [18]:
trainable_params = sum(p.numel() for p in model_2.parameters() if p.requires_grad)
print(trainable_params)

64


In [21]:
n_epochs=300
optimizer = torch.optim.Adam(lr=0.005, params=model_2.parameters())
criterion = nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()

pbar = tqdm(total=len(rp_trainloader), leave=True)
if torch.cuda.is_available():
    devname = "cuda"
else:
    devname = "cpu"
device = torch.device(devname)
model_2.train()
model_2.to(device)
pbar = tqdm(total=len(rp_trainloader), leave=True)

file_name="QRNN_model_12113_rp"
max_val=0
for epoch in range(n_epochs):
    acc=0
    for batch, (feature, label) in enumerate(rp_trainloader):
        optimizer.zero_grad()
        feature, label = feature.to(device), label.to(device)
        preds = model_2(feature).squeeze()
        loss = criterion(preds, label.float())
        preds = preds.ravel()
        label = label.ravel()
        acc += binary_accuracy(preds, label)

        loss.backward()
        optimizer.step()
        pbar.update()
        pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss.cpu().detach()}"
        pbar.refresh()
    acc=acc/(batch+1)
    print('Epoch',epoch,'Training Accuracy:', acc)
    feature, label=next(iter(rp_testloader))
    preds = model_2(feature).squeeze()
    loss = criterion(preds, label.float())
    preds = preds.ravel()
    label = label.ravel()
    acc1 = binary_accuracy(preds, label)
    print('Testing Accuracy:', acc1)
    if max_val< (acc1+acc):
        max_val=acc1+acc
        torch.save(model_2.state_dict(),file_name)
pbar.close()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 0 Training Accuracy: tensor(0.6222)
Testing Accuracy: tensor(0.6452)
Epoch 1 Training Accuracy: tensor(0.6476)
Testing Accuracy: tensor(0.6452)
Epoch 2 Training Accuracy: tensor(0.6730)
Testing Accuracy: tensor(0.6774)
Epoch 3 Training Accuracy: tensor(0.6603)
Testing Accuracy: tensor(0.6774)
Epoch 4 Training Accuracy: tensor(0.6127)
Testing Accuracy: tensor(0.6774)
Epoch 5 Training Accuracy: tensor(0.6381)
Testing Accuracy: tensor(0.6774)
Epoch 6 Training Accuracy: tensor(0.6000)
Testing Accuracy: tensor(0.6774)
Epoch 7 Training Accuracy: tensor(0.6381)
Testing Accuracy: tensor(0.6774)
Epoch 8 Training Accuracy: tensor(0.6349)
Testing Accuracy: tensor(0.6774)
Epoch 9 Training Accuracy: tensor(0.6714)
Testing Accuracy: tensor(0.6774)
Epoch 10 Training Accuracy: tensor(0.6587)
Testing Accuracy: tensor(0.6452)
Epoch 11 Training Accuracy: tensor(0.6460)
Testing Accuracy: tensor(0.6452)
Epoch 12 Training Accuracy: tensor(0.6794)
Testing Accuracy: tensor(0.6452)
Epoch 13 Training Accu

In [22]:
file_name="QRNN_model_12113_rp"
model_2.load_state_dict(torch.load(file_name))
model_2.eval()

qrnn(
  (QRNN): <Quantum Torch Layer: func=circuit>
)

In [23]:
acc=[]
for i in range(100):
    preds = []
    labels = []
    #pbar = tqdm(total=len(mc_testloader), leave=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_2.eval()
    model_2.to(device)
    for batch, (feature, label) in enumerate(rp_testloader):
      feature, label = feature.to(device), label.to(device)
      with torch.no_grad():
        predictions = model_2(feature.float())
        preds.append(predictions.cpu().numpy())
        labels.append(label.cpu().numpy())
      #pbar.update()
      #pbar.desc = f"Batch: {batch}"
      #pbar.refresh()
    #pbar.close()
    preds = torch.tensor(preds).ravel()
    # preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
    labels = torch.tensor(labels).ravel()
    #acc = (preds == labels).sum() / len(preds)
    acc.append(binary_accuracy(preds, labels))
print("Test Accuracy:",np.mean(acc),r"$\pm$",np.std(acc))

Test Accuracy: 0.6451612 $\pm$ 5.9604645e-08


In [25]:
acc=[]
for i in range(100):
    preds = []
    labels = []
    #pbar = tqdm(total=len(rp_trainloader), leave=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_2.eval()
    model_2.to(device)
    for batch, (feature, label) in enumerate(rp_trainloader):
        feature, label = feature.to(device), label.to(device)
        with torch.no_grad():
            predictions = model_2(feature)
            preds=preds+list(predictions.cpu().numpy())
            labels=labels+list(label.cpu().numpy())
      #pbar.update()
      #pbar.desc = f"Batch: {batch}"
      #pbar.refresh()
    #pbar.close()
    preds = torch.tensor(preds).ravel()
    # preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
    labels = torch.tensor(labels).ravel()
    #acc = (preds == labels).sum() / len(preds)
    acc.append(binary_accuracy(preds, labels))
print("Train Accuracy:",np.mean(acc),"with standard error:",np.std(acc))

Train Accuracy: 0.7567569 with standard error: 1.1920929e-07


# Simple TF-IDF vectorization

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, Dataset
from tensorflow import compat
from sklearn.decomposition import PCA

In [88]:
# Custom PyTorch Dataset for data loading preprocessed text
# Input data must be a DataFrame in a particular format. For text one column must be text and other must be label

class txt2vecset(Dataset):
    def __init__(self, data_df, max_seq_length=50, vectorizer=None, pca=None):
        self.df = data_df
        self.max_seq_length = max_seq_length
        self.pca = pca
        if vectorizer is None:
            self.vectorizer = TfidfVectorizer().fit(self.df['text'])
        else:
            self.vectorizer = vectorizer
        self._df2vec()

    def _df2vec(self):
        self.X = self.vectorizer.transform(self.df["text"]).toarray()
        if self.pca is None:
            self.pca = PCA(self.max_seq_length).fit(self.X)
        self.X = self.pca.transform(self.X)

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        label = data['label']
        feature = self.X[idx, :]
        feature, label = torch.acos(torch.tensor(feature)), torch.tensor(label)
        return feature.squeeze(), label.squeeze()
    
    def retattrs(self):
        return self.vectorizer, self.pca

# Model trainer/tester and other helpers

In [89]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

def model_trainer(model, n_epochs, trainloader):
    optimizer = torch.optim.Adam(lr=0.005, params=model.parameters())
    criterion = nn.BCELoss()
    #criterion = torch.nn.CrossEntropyLoss()
    pbar = tqdm(total=len(trainloader), leave=True)
    if torch.cuda.is_available():
        devname = "cuda"
    else:
        devname = "cpu"
    device = torch.device(devname)
    model.train()
    model.to(device)
    max_val=0
    losslst = []
    for epoch in range(n_epochs):
        acc=0
        pbar = tqdm(total=len(trainloader), leave=True)
        for batch, (feature, label) in enumerate(trainloader):
            optimizer.zero_grad()
            feature, label = feature.to(device), label.to(device)
            preds = model(feature.float())
            loss = criterion(preds.flatten(), label.float())
            preds = preds.ravel()
            label = label.ravel()
            acc += binary_accuracy(preds, label)

            loss.backward()
            optimizer.step()
            pbar.update()
            pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss.cpu().detach()}"
            pbar.refresh()
            losslst.append(loss.cpu().detach())
        acc=acc/(batch+1)
        print('Epoch',epoch,'Training Accuracy:', acc)
        pbar.close()
    return model, losslst

# def model_tester(model, testloader):
#     acc=[]
#     for i in range(100):
#         preds = []
#         labels = []
#         pbar = tqdm(total=len(testloader), leave=True)
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         model.eval()
#         model.to(device)
#         for batch, (feature, label) in enumerate(testloader):
#             feature, label = feature.to(device), label.to(device)
#             with torch.no_grad():
#                 predictions = model(feature.float())
#                 preds.append(predictions.cpu().numpy())
#                 labels.append(label.cpu().numpy())
#             pbar.update()
#             pbar.desc = f"Batch: {batch} | Run: {i}"
#             pbar.refresh()
#         pbar.close()
#         preds = torch.tensor(preds).ravel()
#         # preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
#         labels = torch.tensor(labels).ravel()
#         #acc = (preds == labels).sum() / len(preds)
#         acc.append(binary_accuracy(preds, labels))
#     print("Test Accuracy:",np.mean(acc),r"$\pm$",np.std(acc))
#     return preds, acc, labels

def model_tester(model, testloader):
    preds = []
    labels = []
    pbar = tqdm(total=len(testloader), leave=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    for batch, (feature, label) in enumerate(testloader):
        feature, label = feature.to(device), label.to(device)
        with torch.no_grad():
            predictions = model(feature.float())
            preds.append(predictions.cpu())
            labels.append(label.cpu())
        pbar.update()
        pbar.desc = f"Batch: {batch}"
        pbar.refresh()
    pbar.close()
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    rounded_preds = ((torch.round(torch.sign(preds-0.5))+1)//2) #.numpy()
    acc = 1 - (torch.abs(rounded_preds - labels).mean())
    return rounded_preds, acc, labels

## MC RP with TF-IDF

In [90]:
!git clone https://github.com/CQCL/qnlp_lorenz_etal_2021_resources
!mv qnlp_lorenz_etal_2021_resources/datasets mc_rp_dataset

fatal: destination path 'qnlp_lorenz_etal_2021_resources' already exists and is not an empty directory.
mv: cannot stat 'qnlp_lorenz_etal_2021_resources/datasets': No such file or directory


In [91]:
mc_rp_sets_path = Path("mc_rp_dataset")
mc_rp_sets = list(mc_rp_sets_path.glob("*.txt"))
mc_datasets, rp_datasets = list(filter(lambda x: x.name.startswith("mc"), mc_rp_sets)), list(filter(lambda x: x.name.startswith("rp"), mc_rp_sets))

In [92]:
def read_process_mcrp(datapaths: dict):
  def rm(text):
    return " ".join(list(map(lambda x: x[:x.find('_')], text.split())))
  retval = {}
  for datapath in datapaths:
    if "rp" in str(datapath):
      sel = 2
    else:
      sel = 3
    df = pd.DataFrame(list(map(lambda x: [int(x[0]), x[sel:]], datapath.read_text().split("\n"))), columns=['label', 'text'])
    df['text'] = df['text'].apply(rm)
    retval[datapath.name.split(".")[0]] = df
  return retval

def ttsplit(data: pd.DataFrame, test_size=0.2):
    train_data, test_data = train_test_split(data, test_size=test_size, stratify=data['label'], random_state=42)
    train_data, test_data = train_data.reset_index().drop(columns=['index']), test_data.reset_index().drop(columns=['index'])
    # train_data = pd.DataFrame({'text': X_train, 'labels': y_train})
    # test_data = pd.DataFrame({'text': X_test, 'labels': y_test})
    return train_data, test_data

mc_data, rp_data = read_process_mcrp(mc_datasets), read_process_mcrp(rp_datasets)

# # Train test splitting
mc_train = mc_data['mc_train_data']
mc_test = mc_data['mc_test_data']

rp_train = rp_data['rp_train_data']
rp_test = rp_data['rp_test_data']

# mc_train, mc_test = ttsplit(mc_data, 0.2)
# rp_train, rp_test = ttsplit(rp_data, 0.2)

MAX_SEQ_LEN = 4
BATCH_SIZE = 50
NUM_EPOCHS = 200

# TF-IDF vectorization and dataset creation
mc_trainset = txt2vecset(mc_train, MAX_SEQ_LEN)
rp_trainset = txt2vecset(rp_train, MAX_SEQ_LEN)

mc_testset = txt2vecset(mc_test, MAX_SEQ_LEN, *mc_trainset.retattrs())
rp_testset = txt2vecset(rp_test, MAX_SEQ_LEN, *rp_trainset.retattrs())

In [93]:
# MC RP DataLoaders
mc_trainloader = DataLoader(mc_trainset, shuffle=True, batch_size=BATCH_SIZE)
mc_testloader = DataLoader(mc_testset, shuffle=True, batch_size=BATCH_SIZE)

rp_trainloader = DataLoader(rp_trainset, shuffle=True, batch_size=BATCH_SIZE)
rp_testloader = DataLoader(rp_testset, shuffle=True, batch_size=BATCH_SIZE)

In [94]:
# mc training
qrnn_mc_model = qrnn(MAX_SEQ_LEN,1,2,1)
mc_qrnn_trained, mc_train_loss = model_trainer(qrnn_mc_model, NUM_EPOCHS, mc_trainloader)
torch.save(mc_qrnn_trained, "mc_qrnn.pt")
dump(mc_train_loss, "mc_train_loss.joblib")

# loss_plotter(mc_train_loss, NUM_EPOCHS, save=True, save_name="mc_data")


# mc testing
mc_preds, mc_acc, mc_labels = model_tester(mc_qrnn_trained, mc_testloader)
mc_train_preds, mc_train_acc, mc_train_labels = model_tester(mc_qrnn_trained, mc_trainloader)

print(f"mc accuracy test: {mc_acc}")
print(f"mc accuracy train: {mc_train_acc}")

dump(mc_preds, "mc_preds.joblib")
dump(mc_labels, "mc_labels.joblib")
dump(mc_train_preds, "mc_train_preds.joblib")
dump(mc_train_labels, "mc_train_labels.joblib")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0 Training Accuracy: tensor(0.5050)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 Training Accuracy: tensor(0.5300)


...

Epoch 199 Training Accuracy: tensor(0.9700)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

mc accuracy test: 0.03333333507180214
mc accuracy train: 0.02857142873108387


['mc_train_labels.joblib']

In [95]:
# rp training
qrnn_rp_model = qrnn(MAX_SEQ_LEN,1,2,1)
rp_qrnn_trained, rp_train_loss = model_trainer(qrnn_rp_model, NUM_EPOCHS, rp_trainloader)
torch.save(rp_qrnn_trained, "rp_qrnn.pt")
dump(rp_train_loss, "rp_train_loss.joblib")

# loss_plotter(rp_train_loss, NUM_EPOCHS, save=True, save_name="rp_data")


# rp testing
rp_preds, rp_acc, rp_labels = model_tester(rp_qrnn_trained, rp_testloader)
rp_train_preds, rp_train_acc, rp_train_labels = model_tester(rp_qrnn_trained, rp_trainloader)

print(f"rp accuracy test: {rp_acc}")
print(f"rp accuracy train: {rp_train_acc}")

dump(rp_preds, "rp_preds.joblib")
dump(rp_labels, "rp_labels.joblib")
dump(rp_train_preds, "rp_train_preds.joblib")
dump(rp_train_labels, "rp_train_labels.joblib")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0 Training Accuracy: tensor(0.6425)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 Training Accuracy: tensor(0.6758)


...

Epoch 199 Training Accuracy: tensor(0.6342)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

rp accuracy test: 0.3870967626571655
rp accuracy train: 0.3918918967247009


['rp_train_labels.joblib']

# Sentiment Labelled Sentences datasets:
- Amazon
- Yelp
- IMDb

In [ ]:
!wget https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip
!unzip -o sentiment+labelled+sentences.zip
!rm "sentiment labelled sentences/readme.txt"

--2023-08-13 12:31:23--  https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘sentiment+labelled+sentences.zip.1’

sentiment+labelled+     [ <=>                ]  82.21K  --.-KB/s    in 0.07s   

2023-08-13 12:31:23 (1.19 MB/s) - ‘sentiment+labelled+sentences.zip.1’ saved [84188]

Archive:  sentiment+labelled+sentences.zip
  inflating: sentiment labelled sentences/.DS_Store  
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment

In [ ]:
# Separating Amazon, IMDb, Yelp dataset paths
datasets_path = Path("sentiment labelled sentences")
datasets = list(datasets_path.glob("*.txt"))

# Reading all data and creating data loaders from our data loading strategy
def read_process(datapaths: list):
  retval = {}
  for datapath in datapaths:
    df = pd.DataFrame(list(map(lambda x: x.split("\t"), datapath.read_text().split("\n"))), columns=['text', 'label']).dropna()
    df['label'] = df['label'].apply(lambda  x: int(x))
    retval[datapath.name.split(".")[0]] = df
  return retval

def ttsplit(data: pd.DataFrame, test_size=0.2):
    train_data, test_data = train_test_split(data, test_size=test_size, stratify=data['label'], random_state=42)
    train_data, test_data = train_data.reset_index().drop(columns=['index']), test_data.reset_index().drop(columns=['index'])
    # train_data = pd.DataFrame({'text': X_train, 'labels': y_train})
    # test_data = pd.DataFrame({'text': X_test, 'labels': y_test})
    return train_data, test_data

datadict = read_process(datasets)
amazon_data = datadict['amazon_cells_labelled']
imdb_data = datadict['imdb_labelled']
yelp_data = datadict['yelp_labelled']

# Train test splitting
amazon_train, amazon_test = ttsplit(amazon_data, 0.2)
imdb_train, imdb_test = ttsplit(imdb_data, 0.2)
yelp_train, yelp_test = ttsplit(yelp_data, 0.2)


MAX_SEQ_LEN = 4
BATCH_SIZE = 25
NUM_EPOCHS = 200


# TF-IDF vectorization and dataset creation
amazon_trainset = txt2vecset(amazon_train, MAX_SEQ_LEN)
imdb_trainset = txt2vecset(imdb_train, MAX_SEQ_LEN)
yelp_trainset = txt2vecset(yelp_train, MAX_SEQ_LEN)

amazon_testset = txt2vecset(amazon_test, MAX_SEQ_LEN, *amazon_trainset.retattrs())
imdb_testset = txt2vecset(imdb_test, MAX_SEQ_LEN, *imdb_trainset.retattrs())
yelp_testset = txt2vecset(yelp_test, MAX_SEQ_LEN, *yelp_trainset.retattrs())

In [ ]:
# Amazon dataloaders
amazon_trainloader = DataLoader(amazon_trainset, shuffle=True, batch_size=BATCH_SIZE)
amazon_testloader = DataLoader(amazon_testset, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
# Yelp dataloader
yelp_trainloader = DataLoader(yelp_trainset, shuffle=True, batch_size=BATCH_SIZE)
yelp_testloader = DataLoader(yelp_testset, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
# imdb dataloaders
imdb_trainloader = DataLoader(imdb_trainset, shuffle=True, batch_size=BATCH_SIZE)
imdb_testloader = DataLoader(imdb_testset, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
# Amazon training
qrnn_amazon_model = qrnn(MAX_SEQ_LEN,1,2,1)
amazon_qrnn_trained, amazon_train_loss = model_trainer(qrnn_amazon_model, NUM_EPOCHS, amazon_trainloader)
torch.save(amazon_qrnn_trained, "amazon_qrnn.pt")
dump(amazon_train_loss, "amazon_train_loss.joblib")

# loss_plotter(amazon_train_loss, NUM_EPOCHS, save=True, save_name="amazon_data")


# Amazon testing
amazon_preds, amazon_acc, amazon_labels = model_tester(amazon_qrnn_trained, amazon_testloader)
amazon_train_preds, amazon_train_acc, amazon_train_labels = model_tester(amazon_qrnn_trained, amazon_trainloader)

print(f"amazon accuracy test: {amazon_acc}")
print(f"amazon accuracy train: {amazon_train_acc}")

dump(amazon_preds, "amazon_preds.joblib")
dump(amazon_labels, "amazon_labels.joblib")
dump(amazon_train_preds, "amazon_train_preds.joblib")
dump(amazon_train_labels, "amazon_train_labels.joblib")

Epoch 3 Training Accuracy: tensor(0.6125)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 4 Training Accuracy: tensor(0.5600)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 5 Training Accuracy: tensor(0.5625)


...

Epoch 184 Training Accuracy: tensor(0.6288)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 185 Training Accuracy: tensor(0.6313)


  0%|          | 0/32 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# IMDb training
qrnn_imdb_model = qrnn(MAX_SEQ_LEN,1,2,1)
imdb_qrnn_trained, imdb_train_loss = model_trainer(qrnn_imdb_model, NUM_EPOCHS, imdb_trainloader)
torch.save(imdb_qrnn_trained, "imdb_qrnn_s.pt")
dump(imdb_train_loss, "imdb_train_loss.joblib")
# loss_plotter(imdb_train_loss, NUM_EPOCHS, save=True, save_name="imdb_data")


# IMDb testing
imdb_preds, imdb_acc, imdb_labels = model_tester(imdb_qrnn_trained, imdb_testloader)
imdb_train_preds, imdb_train_acc, imdb_train_labels = model_tester(imdb_qrnn_trained, imdb_trainloader)

print(f"imdb accuracy test: {imdb_acc}")
print(f"imdb accuracy train: {imdb_train_acc}")

dump(imdb_preds, "imdb_preds.joblib")
dump(imdb_labels, "imdb_labels.joblib")
dump(imdb_train_preds, "imdb_train_preds.joblib")
dump(imdb_train_labels, "imdb_train_labels.joblib")

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 0 Training Accuracy: tensor(0.4800)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1 Training Accuracy: tensor(0.5050)


...

Epoch 161 Training Accuracy: tensor(0.5425)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 162 Training Accuracy: tensor(0.5625)


  0%|          | 0/32 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# yelp training
qrnn_yelp_model = qrnn(MAX_SEQ_LEN,1,2,1)
yelp_qrnn_trained, yelp_train_loss = model_trainer(qrnn_yelp_model, NUM_EPOCHS, yelp_trainloader)
torch.save(yelp_qrnn_trained, "yelp_qrnn.pt")
dump(yelp_train_loss, "yelp_train_loss.joblib")
# loss_plotter(yelp_train_loss, NUM_EPOCHS, save=True, save_name="yelp_data")


# Yelp testing
yelp_preds, yelp_acc, yelp_labels = model_tester(yelp_qrnn_trained, yelp_testloader)
yelp_train_preds, yelp_train_acc, yelp_train_labels = model_tester(yelp_qrnn_trained, yelp_trainloader)

print(f"yelp accuracy test: {yelp_acc}")
print(f"yelp accuracy train: {yelp_train_acc}")

dump(yelp_preds, "yelp_preds.joblib")
dump(yelp_labels, "yelp_labels.joblib")
dump(yelp_train_preds, "yelp_train_preds.joblib")
dump(yelp_train_labels, "yelp_train_labels.joblib")

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 0 Training Accuracy: tensor(0.5125)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1 Training Accuracy: tensor(0.5050)


...

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 131 Training Accuracy: tensor(0.5838)


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 132 Training Accuracy: tensor(0.6100)


  0%|          | 0/32 [00:00<?, ?it/s]